In [0]:

!gdown  https://drive.google.com/uc?id=1fpT0vqOrDDxiTKrlrdlf2_PZRfBFuHxP

!unzip dogs-vs-cats-for-pytorch.zip



In [0]:
import torch, os
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets, transforms

In [0]:
# prepare data

BATCH_SIZE      = 32
IMG_SIZE        = 224

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
)

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])



transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

train_data = datasets.ImageFolder('cat_dog_data/Cat_Dog_data/train', transform=transform_train)
trainloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_data = datasets.ImageFolder('cat_dog_data/Cat_Dog_data/test', transform=transform_test)
testloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [0]:

from inception import inception_v3

# class
classes = {0: 'cat', 1: 'dog'}

net = inception_v3(pretrained=False, num_classes=len(classes))
final_conv = 'Mixed_7c'

net.cuda()

In [0]:
from train import *
LEARNING_RATE   = 0.01
EPOCH           = 20
USE_CUDA=True
# retrain
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)



for epoch in range (1, EPOCH + 1):
  for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        break


In [0]:
# hook the feature extractor
features_blobs = []

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get(final_conv).register_forward_hook(hook_feature)


# CAM
if CAM:
    root = 'sample.jpg'
    img = Image.open(root)
    get_cam(net, features_blobs, img, classes, root)